In [54]:
import requests
import pandas as pd

class PokemonExtract:
    def __init__(self):
        self.api_url = "https://pokeapi.co/api/v2/pokemon"
        self.offset = 0
        self.limit = 100  # Escolhi um número menor para facilitar testes
        self.total_pokemon = 0
        self.pokemon_data = []

    def get_pokemon_names(self, offset, limit):
        base_url = f"{self.api_url}?offset={offset}&limit={limit}"
        response = requests.get(base_url)

        if response.status_code == 200:
            data = response.json()
            return [pokemon['name'] for pokemon in data['results']]
        else:
            print(f"Error: Could not retrieve the list of Pokémon. Status code: {response.status_code}")
            return []
        
    def get_pokemon_data(self, pokemon_name):
        base_url = f"{self.api_url}/{pokemon_name.lower()}"
        response = requests.get(base_url)

        if response.status_code == 200:
            data = response.json()
            return {
                "name": data['name'].capitalize(),
                "height": data['height'],
                "weight": data['weight'],
                "types": ", ".join([type_info['type']['name'].capitalize() for type_info in data['types']]),
                "abilities": ", ".join([ability['ability']['name'].capitalize() for ability in data['abilities']])
            }
        else:
            print(f"Error: Could not retrieve data for {pokemon_name}. Status code: {response.status_code}")
            return None
    
    def paginacao(self):
        while len(self.pokemon_data) < 10: # Coloquei um limite aqui, porque estava demorando muito para carregar
            while True:
                pokemon_names = self.get_pokemon_names(self.offset, self.limit)

#                if not pokemon_names:
#                    break

                for pokemon in pokemon_names:
                    data = self.get_pokemon_data(pokemon)
                    if data:
                        self.pokemon_data.append(data)

                self.offset += self.limit
                self.total_pokemon += len(pokemon_names)
                print(f"Processed {self.total_pokemon} Pokémon so far...")

                df = pd.DataFrame(self.pokemon_data)
                return df
    
    def formatTable(self, df, fileName):
        df.to_excel(f'{fileName}.xlsx', index=False)
        print(f'Arquivo {fileName}.xlsx gerado com sucesso')

In [55]:
pokemon_extract = PokemonExtract()


In [56]:
df_pokemon = pokemon_extract.paginacao()


Processed 100 Pokémon so far...


In [57]:
print(df_pokemon)

          name  height  weight          types  \
0    Bulbasaur       7      69  Grass, Poison   
1      Ivysaur      10     130  Grass, Poison   
2     Venusaur      20    1000  Grass, Poison   
3   Charmander       6      85           Fire   
4   Charmeleon      11     190           Fire   
..         ...     ...     ...            ...   
95     Drowzee      10     324        Psychic   
96       Hypno      16     756        Psychic   
97      Krabby       4      65          Water   
98     Kingler      13     600          Water   
99     Voltorb       5     104       Electric   

                                 abilities  
0                    Overgrow, Chlorophyll  
1                    Overgrow, Chlorophyll  
2                    Overgrow, Chlorophyll  
3                       Blaze, Solar-power  
4                       Blaze, Solar-power  
..                                     ...  
95         Insomnia, Forewarn, Inner-focus  
96         Insomnia, Forewarn, Inner-focus  
97  Hy

In [58]:
pokemon_extract.formatTable(df_pokemon, 'pokemon_table')

Arquivo pokemon_table.xlsx gerado com sucesso
